## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-01-23-49-11 +0000,nypost,Dentist and wife found fatally shot inside Ohi...,https://nypost.com/2026/01/01/us-news/dentist-...
1,2026-01-01-23-46-25 +0000,nypost,"French soccer player, 19, badly burned in dead...",https://nypost.com/2026/01/01/world-news/frenc...
2,2026-01-01-23-43-00 +0000,wsj,Gold Rises Amid Geopolitical Risks,https://www.wsj.com/finance/commodities-future...
3,2026-01-01-23-40-49 +0000,bbc,"Around 40 dead in Swiss ski resort bar fire, p...",https://www.bbc.com/news/articles/cx2pmyyeg78o...
4,2026-01-01-23-10-19 +0000,nypost,Trump tears into ‘average guy’ George Clooney ...,https://nypost.com/2026/01/01/us-news/trump-te...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2353/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
68,new,45
69,year,41
33,trump,27
20,fire,18
18,swiss,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
118,2026-01-01-11-13-29 +0000,cbc,"Around 40 dead, more than 100 injured in blaze...",https://www.cbc.ca/news/world/bar-fire-switzer...,161
102,2026-01-01-13-52-12 +0000,nypost,Horrific video shows the moment deadly New Yea...,https://nypost.com/2026/01/01/world-news/new-y...,159
8,2026-01-01-22-55-18 +0000,wapo,"About 40 killed, 115 injured in New Year’s bar...",https://www.washingtonpost.com/world/2026/01/0...,158
151,2026-01-01-07-42-14 +0000,nypost,"40 people dead, 100 hurt after fire rips throu...",https://nypost.com/2026/01/01/world-news/swiss...,157
79,2026-01-01-17-09-57 +0000,nypost,Foreign tourists among 47 confirmed dead in Ne...,https://nypost.com/2026/01/01/world-news/many-...,146


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
118,161,2026-01-01-11-13-29 +0000,cbc,"Around 40 dead, more than 100 injured in blaze...",https://www.cbc.ca/news/world/bar-fire-switzer...
111,58,2026-01-01-12-48-00 +0000,wsj,President Trump delayed by a year tariff incre...,https://www.wsj.com/economy/trade/trump-rolls-...
47,52,2026-01-01-20-02-35 +0000,nypost,‘Tax the rich’ chant breaks out as Bernie Sand...,https://nypost.com/2026/01/01/us-news/tax-the-...
1,41,2026-01-01-23-46-25 +0000,nypost,"French soccer player, 19, badly burned in dead...",https://nypost.com/2026/01/01/world-news/frenc...
48,32,2026-01-01-19-55-28 +0000,nypost,Son of late New York musician was ready to com...,https://nypost.com/2026/01/01/us-news/grieving...
87,31,2026-01-01-15-40-43 +0000,nyt,Trump Must Return Command of California Nation...,https://www.nytimes.com/2025/12/31/us/trump-na...
171,29,2026-01-01-02-29-38 +0000,nyt,CI.A. Says Ukraine Did Not Target Putin’s Home...,https://www.nytimes.com/2025/12/31/us/politics...
23,28,2026-01-01-21-43-19 +0000,nypost,HHS audit for 2012 found $16M in ‘improper’ ch...,https://nypost.com/2026/01/01/us-news/hhs-audi...
182,25,2026-01-01-01-30-11 +0000,nypost,"US forces obliterate five alleged drug boats, ...",https://nypost.com/2025/12/31/world-news/3-kil...
42,24,2026-01-01-20-24-13 +0000,nypost,"Hit-and-run driver kills one NYC pedestrian, l...",https://nypost.com/2026/01/01/us-news/hit-and-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
